In [1]:
import os
import argparse
from pathlib import Path
import warnings
from conf import settings
from utils import get_test_dataloader

import sys
sys.path.insert(0,'..')

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from autoattack import AutoAttack
from models.resnet import resnet18

In [2]:
# load model
model = resnet18()
ckpt = torch.load('18_on_100.pth')
model.load_state_dict(ckpt)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
# Set the model to evaluation mode
model.eval()

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2_x): Sequential(
    (0): BasicBlock(
      (residual_function): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (residual_function): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_

# Linf

In [3]:
# load data
aa_data_linf  = torch.load("aa_results/aa_cifar100_standard_1_5000_norm_Linf.pth")

# Define the transform for the test data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5088964127604166, 0.48739301317401956, 0.44194221124387256),
                          (0.2682515741720801, 0.2573637364478126, 0.2770957707973042))])
# Load the CIFAR-100 test set
testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)

Files already downloaded and verified


In [12]:
testset

Dataset CIFAR100
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5088964127604166, 0.48739301317401956, 0.44194221124387256), std=(0.2682515741720801, 0.2573637364478126, 0.2770957707973042))
           )

In [11]:
images = aa_data_linf['adv_complete'][0]
labels = aa_data_linf['adv_complete'][1].tolist()
aa_list_linf = []

for i in range(len(images)):
    sample = (images[i], labels[i])
    aa_list_linf.append(sample)


In [12]:
# let the label of aa samples be 100
real_aa_list_linf = []

for i in range(len(aa_list_linf)):
    if aa_list_linf[i][1] != testset[i][1]:
        real_aa_sample = (images[i], 100)
        real_aa_list_linf.append(real_aa_sample)

In [13]:
len(real_aa_list_linf)

4808

In [15]:
torch.save({'adv_complete': real_aa_list_linf}, 'real_cifar100_Linf_5000.pth')

In [46]:
# #combined_items
# img = aa_list[1][0]
# batched_image = img.unsqueeze(0)
# l = aa_list[0][1]
# batched_image = batched_image.to(device)

# output = model(batched_image)
# output
# _, predicted = output.max(1)
# predicted.eq(l)

In [35]:
# def get_real_aa(model, aa_list):
#     misclassified_items = [] 
#     for sample in aa_list:
#         (image, label) = sample
#         batched_image = image.unsqueeze(0)
#         batched_image = batched_image.to(device)
#         output = model(batched_image)
#         _, predicted = output.max(1)
#         if predicted.eq(label) == False:
#             misclassified_items.append((batched_image.cpu(), 10))
#     return tuple(misclassified_items)

In [36]:
# misclassified_items = get_real_aa(model, aa_list)

In [63]:
# len(misclassified_items)

# L2

In [18]:
# similar with Linf
aa_data_l2 = torch.load("aa_results/aa_cifar100_standard_1_5000_norm_L2.pth")

In [19]:
images = aa_data_l2['adv_complete'][0]
labels = aa_data_l2['adv_complete'][1].tolist()
aa_list_l2 = []

for i in range(len(images)):
    sample = (images[i], labels[i])
    aa_list_l2.append(sample)

In [20]:
real_aa_list_l2 = []

for i in range(len(aa_list_l2)):
    if aa_list_l2[i][1] != testset[i][1]:
        real_aa_sample = (images[i], 100)
        real_aa_list_l2.append(real_aa_sample)

In [21]:
len(real_aa_list_l2)

3784

In [23]:
torch.save({'adv_complete': real_aa_list_l2}, 'real_cifar100_L2_5000.pth')